In [11]:
import cv2
import numpy as np
from scipy.spatial import distance

# Load both images, convert to double and to grayscale
img1 = cv2.imread('left.jpg',0)  # Image 1
img2 = cv2.imread('right.jpg',0)  # Image 2

# Detect feature points in both images
sift = cv2.xfeatures2D.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# Compute distances between every descriptor in one image and every descriptor in the other image
distances = distance.cdist(des1, des2, 'sqeuclidean')

# Select putative matches based on the matrix of pairwise descriptor distances
matches = distances < 200  # Threshold can be adjusted

# Implement RANSAC to estimate a homography mapping one image onto the other
src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()

# Display the locations of inlier matches in both images
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

img3 = cv2.drawMatches(img1,kp1,img2,kp2,matches,None,**draw_params)

cv2.imshow('Matches', img3)
cv2.waitKey(0)
cv2.destroyAllWindows()


AttributeError: module 'cv2' has no attribute 'xfeatures2D'